In [54]:
import numpy as np
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from seqeval.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
from lime.lime_text import LimeTextExplainer

import sys

sys.path.insert(1, '../scripts/')
from ner_wrapper import NERWrapper

In [55]:
# --- STEP 1: Define Labels ---
label_list = ["O", "B-I-Product", "B-B-Product", "I-B-Product", "I-I-Product",
               "B-B-PRICE", "B-I-PRICE", "B-B-LOC", "B-I-LOC", "I-I-LOC"]
label_to_id = {label: idx for idx, label in enumerate(label_list)}
id_to_label = {idx: label for label, idx in label_to_id.items()}

In [56]:
# --- STEP 2: Load CoNLL-style text files ---
def read_conll(filepath):
    with open(filepath, encoding="utf-8") as f:
        tokens, tags, samples = [], [], []
        for line in f:
            line = line.strip()
            if line == "":
                if tokens:
                    samples.append({"tokens": tokens, "ner_tags": tags})
                    tokens, tags = [], []
            else:
                splits = line.split()
                # print(splits)
                tokens.append(splits[0])
                tags.append(label_to_id.get(splits[3], 0))
        if tokens:
            samples.append({"tokens": tokens, "ner_tags": tags})
    return samples

In [57]:
train_data = read_conll("../../data/sample_label.txt")  
valid_data = read_conll("../../data/sample_label.txt")

dataset = DatasetDict({
    "train": Dataset.from_list(train_data),
    "validation": Dataset.from_list(valid_data)
})

In [58]:
# --- STEP 3: Load tokenizer ---
model_checkpoint = "rasyosef/bert-tiny-amharic"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [59]:
# --- STEP 4: Tokenize and align labels ---
def tokenize_and_align_labels(example):
    tokenized_inputs = tokenizer(example["tokens"], truncation=True, is_split_into_words=True)
    word_ids = tokenized_inputs.word_ids()
    aligned_labels = []
    prev_word_id = None
    for word_id in word_ids:
        if word_id is None:
            aligned_labels.append(-100)
        elif word_id != prev_word_id:
            aligned_labels.append(example["ner_tags"][word_id])
        else:
            # If subword, mark as I-*
            label = example["ner_tags"][word_id]
            if label == 0:
                aligned_labels.append(0)
            else:
                aligned_labels.append(label)
        prev_word_id = word_id
    tokenized_inputs["labels"] = aligned_labels
    return tokenized_inputs

In [60]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=False)
# --- STEP 5: Load model ---
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

# --- STEP 6: Define training arguments ---
training_args = TrainingArguments(
    output_dir="./models",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_strategy="epoch",
    save_strategy="no",
    eval_strategy="no",
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

Map: 100%|██████████| 9/9 [00:00<00:00, 639.10 examples/s]
Some weights of BertForTokenClassification were not initialized from the model checkpoint at rasyosef/bert-tiny-amharic and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [61]:
# --- STEP 7: Define metrics ---
def compute_metrics(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=2)

    true_labels = [[id_to_label[l] for l in label if l != -100] for label in labels]
    true_preds = [[id_to_label[p] for (p, l) in zip(preds, label) if l != -100]
                  for preds, label in zip(predictions, labels)]

    return {
        "precision": precision_score(true_labels, true_preds),
        "recall": recall_score(true_labels, true_preds),
        "f1": f1_score(true_labels, true_preds),
        "accuracy": accuracy_score(true_labels, true_preds)
    }

In [62]:
# --- STEP 8: Fine-tune the model ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForTokenClassification(tokenizer),
    compute_metrics=compute_metrics
)

/tmp/ipykernel_531237/2677630316.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [63]:
trainer.train()

Step,Training Loss
1,2.373900
2,2.361100
3,2.347000
4,2.335300


TrainOutput(global_step=4, training_loss=2.3543407917022705, metrics={'train_runtime': 0.8198, 'train_samples_per_second': 43.911, 'train_steps_per_second': 4.879, 'total_flos': 27687955680.0, 'train_loss': 2.3543407917022705, 'epoch': 4.0})

In [64]:
# After predictions
output = trainer.predict(tokenized_dataset["validation"])
predictions = output.predictions
labels = output.label_ids
preds = np.argmax(predictions, axis=2)

In [65]:
true_labels = [[id_to_label[l] for l in label if l != -100] for label in labels]
true_preds = [[id_to_label[p] for (p, l) in zip(pred, label) if l != -100]
              for pred, label in zip(preds, labels)]
# Generate classification report
report = classification_report(true_labels, true_preds)

print(report)

           precision    recall  f1-score   support

  Product       0.03      0.17      0.05        95
    PRICE       0.00      0.05      0.01        21
      LOC       0.06      0.24      0.10       143

micro avg       0.04      0.20      0.07       259
macro avg       0.05      0.20      0.07       259



In [ ]:
# Initialize
ner_model = NERWrapper(model, tokenizer)
explainer = LimeTextExplainer(class_names=label_list)

text = "ዋጋ 1000 ብር በአዲስ አበባ ይሸጣል።"
# Explain
exp = explainer.explain_instance(text, ner_model.predict_proba, num_features=10)
exp.show_in_notebook()

NameError: name 'np' is not defined